In [3]:
import tensorflow as tf
import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,LSTM,Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [4]:
#using google colab to create the program ,therefore mounting the google drive to extract the relevant files
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,LSTM,Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [35]:
#extracting the english and french sentences from the fra.txt file to create input files with english sentences and corresponding output files with french sentences
import os
l=os.listdir('/content/drive/My Drive/')
input_text=[]
target_text=[]
val_in=[]
val_out=[]
with open('/content/drive/My Drive/fra.txt') as d:
  l=d.readlines()
  for row in  l:
    ipt,tgr,_=row.split('\t')
    input_text.append(ipt)
    target_text.append(tgr)
  
print(len(input_text) )
print(len(target_text) )
  
  
  

178009
178009


In [ ]:
#data cleaning : removing punctuation marks from the sentences of bit the languages
import string 
f1=[t.translate(str.maketrans('','',string.punctuation)) for t in input_text]
f2=[t.translate(str.maketrans('','',string.punctuation)) for t in target_text]

input_text=f1
target_text=f2

input_text

In [37]:
# creating tokenizers to tokenize the sentences and convert them to corresponding sentences
tk1=Tokenizer(oov_token='<UNK>')
tk2=Tokenizer(oov_token='<UNK>')
inpt=[]
trgt=[]
tk1.fit_on_texts(input_text)
tk2.fit_on_texts(target_text)

In [ ]:
#considering maximum sentence length equal to 10 for english and french sentences
l1=10
l2=10

In [39]:
#funtion to convert the text to sequences of numbers and padding those sequences to
def to_encode(tkn,arr,le):
  inpt=tkn.texts_to_sequences(arr)
  data=pad_sequences(inpt,maxlen=le,padding='post')
  return data


In [40]:
#considering first 100000 entries of the data for training the data and using rest of them for testing and prediction
val_in=input_text[100000:]
val_out=target_text[100000:]
input_text=input_text[:100000]
output_text=target_text[:100000]

inpt=to_encode(tk1,input_text,l1)
trgt=to_encode(tk2,output_text,l2)
val_in=to_encode(tk1,val_in,l1)
val_out=to_encode(tk2,val_out,l2)

In [42]:
#importing modules to create the LSTM model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,Dense,LSTM,Bidirectional,RepeatVector
from tensorflow.keras import optimizers
#encoder_input=Embedding(len(tk1.word_counts),100,input_length=l1)

In [45]:
trgt.shape

(100000, 10)

In [13]:
#creating the encoder and decoder LSTM layers of our prediction model
model=Sequential()
model.add(Embedding(len(tk1.word_counts),100,input_length=l1))
model.add(LSTM(100))
model.add(RepeatVector(l2))
model.add(LSTM(100,return_sequences=True))

model.add(Dense(len(tk2.word_counts),activation='softmax'))


In [14]:
#compiling our model
rms=optimizers.Adam(lr=0.01)
model.compile(optimizer=rms,loss='sparse_categorical_crossentropy')


In [15]:
#training the model on our training data
model.fit(inpt,trgt.reshape(trgt.shape[0],trgt.shape[1],1),epochs=10,validation_split=0.2)

Epoch 1/10
2500/2500 [==============================] - 146s 58ms/step - loss: 4.0234 - val_loss: 4.5425
Epoch 2/10
2500/2500 [==============================] - 144s 58ms/step - loss: 3.3063 - val_loss: 4.3204
Epoch 3/10
2500/2500 [==============================] - 142s 57ms/step - loss: 3.0840 - val_loss: 4.1196
Epoch 4/10
2500/2500 [==============================] - 146s 58ms/step - loss: 2.9256 - val_loss: 3.9645
Epoch 5/10
2500/2500 [==============================] - 143s 57ms/step - loss: 2.7777 - val_loss: 3.8964
Epoch 6/10
2500/2500 [==============================] - 141s 57ms/step - loss: 2.6687 - val_loss: 3.8228
Epoch 7/10
2500/2500 [==============================] - 143s 57ms/step - loss: 2.5873 - val_loss: 3.7980
Epoch 8/10
2500/2500 [==============================] - 143s 57ms/step - loss: 2.5357 - val_loss: 3.7993
Epoch 9/10
2500/2500 [==============================] - 142s 57ms/step - loss: 2.4864 - val_loss: 3.7533
Epoch 10/10
2500/2500 [==============================] 

In [16]:
#predicting the output for the val_in data prepared in earlier steps
predicted=model.predict_classes((val_in.reshape(val_in.shape[0],val_in.shape[1]))[:100])

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [47]:
#creating the list of (word,index) pairs in the french language
jk=tk2.word_index.items()
jk=list(jk)
print(jk)

[('<UNK>', 1), ('je', 2), ('de', 3), ('pas', 4), ('que', 5), ('à', 6), ('ne', 7), ('la', 8), ('le', 9), ('vous', 10), ('il', 11), ('tom', 12), ('est', 13), ('un', 14), ('ce', 15), ('tu', 16), ('a', 17), ('nous', 18), ('en', 19), ('une', 20), ('les', 21), ('jai', 22), ('me', 23), ('suis', 24), ('pour', 25), ('faire', 26), ('cest', 27), ('elle', 28), ('ça', 29), ('dans', 30), ('plus', 31), ('des', 32), ('qui', 33), ('tout', 34), ('te', 35), ('ma', 36), ('avec', 37), ('mon', 38), ('fait', 39), ('du', 40), ('veux', 41), ('se', 42), ('au', 43), ('si', 44), ('et', 45), ('quil', 46), ('cette', 47), ('y', 48), ('sont', 49), ('son', 50), ('très', 51), ('votre', 52), ('être', 53), ('pourquoi', 54), ('dit', 55), ('cela', 56), ('temps', 57), ('sur', 58), ('pense', 59), ('été', 60), ('moi', 61), ('était', 62), ('lui', 63), ('peux', 64), ('nest', 65), ('ils', 66), ('chose', 67), ('jamais', 68), ('nai', 69), ('sais', 70), ('ici', 71), ('comment', 72), ('où', 73), ('vraiment', 74), ('bien', 75), ('ton

In [48]:
# function to find the word corresponding to a particualr index in the output sequence because the output of our model is in the form of numerical sequences
def get_w(ind,tk):
  #print(di.get())
  for val in tk:
   # print(val," ")
    if val[1]==ind:
      return val[0]
  return None


In [50]:
#function to create the corresponding sentences form the output sequences obtained from the prediction of our model
pred_text=[]
for i in predicted:
  ls=[]
  for j in range(len(i)):
    t=get_w(i[j],jk)
    if j>0:
      if (t==get_w(i[j-1],jk)) or t==None:
        ls.append('')
      else:
        ls.append(t)
    else:
      if t==None:
        ls.append('')
      else:
        ls.append(t)
  pred_text.append(' '.join(ls))


In [53]:
#creating a sample dataframe to check the amount of similarity between the actual(ac) sentences and the predicted sentences(pre)
import pandas as pd
df=pd.DataFrame({'ac':target_text[100000:100100],'pre':pred_text})
df

,ac,pre
0,Elle soccupe de mes enfants,elle a très
1,Elle a parlé avec le président,elle est
2,Elle a parlé avec la présidente,elle est
3,Elle réfléchit pendant quelques minutes,il a un
4,Elle lui dit quelle était triste,il a dit
...,...,...
95,Alors de quoi étionsnous en train de parler,questce que vous
96,Certaines espèces doiseaux ne peuvent pas voler,ne pas
97,Quelques personnes sont toujours inquiètes,mon sont
98,Il y a des gens qui croient aux fantômes,elles sont
